<a href="https://colab.research.google.com/github/PatrickAllenCooper/learning-resources/blob/master/Fast_Gradient_Attacks_By_Hand_For_Tabular_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are making use of the loan dataset https://www.kaggle.com/deyritwik/bank-loan

Download the csv version of this dataset, unpack it, and drop it in the root directory of this collab notebook.

The objective here is to determine whether some individual will be approved for a loan based on the details contained within their bio.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense

from random import randrange

# dataset import
dataset = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
dataset.head(10)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


Above we see a snapshot of the data we're going to be working with. Our training data will be the columns 'Age'-'Mortgage', while 'Personal Loan' is the label being used.

Let's grab numpy arrays from our csv.

In [ ]:
# allocate csv to numpy arrays
x = dataset.iloc[:,1:9].values
y = dataset.iloc[:,9:10].values


print(y)
# normalize the data
sc = StandardScaler()
x = sc.fit_transform(x)

# one hot encode the data
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

# split the data to train and test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1)


[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


Finally we need to define and train the model we will later be fooling.

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=100, batch_size=64)


Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 0.4235 - accuracy: 0.8931
Epoch 2/100
71/71 [==============================] - 0s 987us/step - loss: 0.2292 - accuracy: 0.9078
Epoch 3/100
71/71 [==============================] - 0s 942us/step - loss: 0.1764 - accuracy: 0.9309
Epoch 4/100
71/71 [==============================] - 0s 995us/step - loss: 0.1415 - accuracy: 0.9520
Epoch 5/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1195 - accuracy: 0.9611
Epoch 6/100
71/71 [==============================] - 0s 1ms/step - loss: 0.1063 - accuracy: 0.9664
Epoch 7/100
71/71 [==============================] - 0s 990us/step - loss: 0.0983 - accuracy: 0.9689
Epoch 8/100
71/71 [==============================] - 0s 961us/step - loss: 0.0923 - accuracy: 0.9698
Epoch 9/100
71/71 [==============================] - 0s 982us/step - loss: 0.0875 - accuracy: 0.9720
Epoch 10/100
71/71 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.97

Not bad in terms of the performance!

No doubt we could do better, but this will serve our purposes.

Now we begin work on the adversarial learner. First we must isolate a particular sample we are interested in perturbating. For the sake of simplicity, let's make it the first one.


In [ ]:
x_perb = x[0:1]
y_perb = y[0:1]

print("Sample values:")
print(x_perb)
print(y_perb)

predicted_value = model.predict(x_perb)
print("Predicted value:", predicted_value)


Sample values:
[[-1.77441684 -1.66607847 -0.53822878 -0.96411408  1.39741398 -0.19338544
  -1.04907792 -0.55552393]]
[[1. 0.]]
Predicted value: [[9.9999726e-01 2.7606277e-06]]


We see that our model is very close to correct in its prediction. Now let's defeat this apparent accuracy by using FGSM.

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_x, input_y):
  input_x = tf.convert_to_tensor(input_x)
  input_y = tf.convert_to_tensor(input_y)
  with tf.GradientTape() as tape:
    tape.watch(input_x)
    prediction = model(input_x)
    loss = loss_object(input_y, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_x)
  print(gradient)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

perturbations = create_adversarial_pattern(x_perb, y_perb)
print(perturbations)

tf.Tensor(
[[-1.46047851e-05  1.04717983e-05  1.91687195e-05  1.21514938e-06
  -7.00618466e-06  1.97321478e-05 -6.87818283e-06  2.26691282e-06]], shape=(1, 8), dtype=float64)
tf.Tensor([[-1.  1.  1.  1. -1.  1. -1.  1.]], shape=(1, 8), dtype=float64)


Great! Now we have our perturbations. Now all that remains is to apply various epsilons to see the degree to which we need to effect the original feature vector in order to create a difference in classification.

In [ ]:
epsilons = [0, 0.01, 0.1, 0.15, 0.30, 0.45, 0.60, 0.75, 1, 2]

for i, eps in enumerate(epsilons):
  adv_x = x_perb + eps*perturbations
  print("Epsilon", eps, ":", model.predict(adv_x))

Epsilon 0 : [[9.9999726e-01 2.7606277e-06]]
Epsilon 0.01 : [[9.9999630e-01 3.7065993e-06]]
Epsilon 0.1 : [[9.999474e-01 5.256127e-05]]
Epsilon 0.15 : [[9.9977070e-01 2.2931358e-04]]
Epsilon 0.3 : [[0.9922616  0.00773849]]
Epsilon 0.45 : [[0.9834489  0.01655106]]
Epsilon 0.6 : [[0.98010397 0.01989603]]
Epsilon 0.75 : [[0.9906695  0.00933043]]
Epsilon 1 : [[0.99779224 0.00220777]]
Epsilon 2 : [[1.000000e+00 1.545734e-08]]


We have verififed a relatively high level of resistance to adversarial examples in our results. You can repeat this with a variety of different entries and confirm the results across the dataset.